In [6]:
import pandas as pd

In [7]:
df = pd.read_pickle('../data/data_w_descs.pkl')
df.head(2)

,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,OnTime_Status,CASE_STATUS,CLOSURE_REASON,CASE_TITLE,SUBJECT,REASON,...,Property_Type,Property_ID,LATITUDE,LONGITUDE,Source,Geocoded_Location,case_enquiry_id,description,specific_location,title
0,101000958209,11/01/2013 09:27:19 AM,11/15/2013 09:27:19 AM,11/27/2013 10:15:45 AM,OVERDUE,Closed,Case Closed Case Resolved,Sign Repair,Transportation - Traffic Division,Signs & Signals,...,Intersection,4053.0,42.3537,-71.0580,Constituent Call,"(42.3537, -71.058)",1.010010e+11,Public safety sign: No | Type of sign: no stop...,None,Damaged Sign at Intersection of Devonshire St ...
1,101001243239,01/06/2015 03:26:49 PM,01/21/2015 03:26:00 PM,01/12/2015 10:49:02 PM,ONTIME,Closed,Case Closed Case Resolved replaced LED. jouber...,outage///printed,Public Works Department,Street Lights,...,Address,63245.0,42.3594,-71.0587,Employee Generated,"(0, 0)",1.010012e+11,Electric or gas light: Electric,None,"Street Lights at 0 Geneva Ave, Dorchester"


In [8]:
df.neighborhood.drop_duplicates()

0                        Downtown / Financial District
1                                           Dorchester
2                                                  NaN
4                                           Roslindale
5                                   Allston / Brighton
6                                          East Boston
8                                          Charlestown
9                                              Roxbury
11                                                 NaN
15                                           Hyde Park
16                                        West Roxbury
17                                       Jamaica Plain
25                                        Mission Hill
29                                                 NaN
31        Fenway / Kenmore / Audubon Circle / Longwood
33                                    Greater Mattapan
39                                        South Boston
40              South Boston / South Boston Waterfront
45        

In [5]:
df.shape

(905650, 37)

## Adding census tract to df

In [1]:
import shapegeocode

In [2]:
def get_tract_block_group(lat, long_, shp_obj):
    d = gc.geocode(lat, long_)
    return d['TRACTCE'] + d['BLKGRPCE']

In [3]:
gc = shapegeocode.geocoder('../data/shape-files/cb_2015_25_bg_500k.shp')

## >>>

In [7]:
gc.geocode(42.3594, -71.0587)

{'AFFGEOID': '1500000US250250303003',
 'ALAND': 304419,
 'AWATER': 0,
 'BLKGRPCE': '3',
 'COUNTYFP': '025',
 'GEOID': '250250303003',
 'LSAD': 'BG',
 'NAME': '3',
 'STATEFP': '25',
 'TRACTCE': '030300'}

## <<<

In [10]:
df.shape

(905650, 37)

In [12]:
df['tract_and_block_group'] = df[['LATITUDE', 'LONGITUDE']].apply(lambda row: get_tract_block_group(row['LATITUDE'], row['LONGITUDE'], gc), axis=1)

In [ ]:
df.tract_and_block_group.isnull().sum()

## Now adding census data to df

In [3]:
df_census = pd.read_pickle('../data/census_data_aggregated.pkl')
df_census.head(1).T

,0
tract_and_block_group,0001001
bedroom_total_ppl,560
bedroom_0,8
bedroom_1,120
bedroom_2,193
bedroom_3,183
bedroom_4,56
bedroom_5+,0
school_total,960
school_0_none,23


In [5]:
df_census[df_census.tract_and_block_group == '9817001'].T

,643
tract_and_block_group,9817001
bedroom_total_ppl,0
bedroom_0,0
bedroom_1,0
bedroom_2,0
bedroom_3,0
bedroom_4,0
bedroom_5+,0
school_total,0
school_0_none,0


In [ ]:
df_final = df.merge(df_census, on='tract_and_block_group', how='left')
df_final.head(1).T

In [ ]:
df.shape, df_census.shape, df_final.shape

In [ ]:
from datetime import datetime
to_datetime = lambda d: datetime.strptime(d, '%m/%d/%Y %H:%M:%S %p') if type(d) != float else pd.np.NaN

In [ ]:
df['OPEN_DT'] = df['OPEN_DT'].apply(lambda datetime_text: to_datetime(datetime_text))
df['CLOSED_DT'] = df['CLOSED_DT'].apply(lambda datetime_text: to_datetime(datetime_text))
df['TARGET_DT'] = df['TARGET_DT'].apply(lambda datetime_text: to_datetime(datetime_text))

In [5]:
df['COMPLETION_TIME'] = (df.CLOSED_DT - df.OPEN_DT).apply(lambda x: x / pd.np.timedelta64(1, 'h'))

In [ ]:
!ls -lh *.h5

In [9]:
df_final.to_hdf('../data/data_w_descs_and_census.h5', key='data_w_descs_and_census', format='fixed')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_items] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:1101: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['OnTime_Status', 'CASE_STATUS', 'CLOSURE_REASON', 'CASE_TITLE', 'SUBJECT', 'REASON', 'TYPE', 'QUEUE', 'Department', 'SubmittedPhoto', 'ClosedPhoto', 'Location', 'pwd_district', 'police_district', 'neighborhood',

In [6]:
df[df.OPEN_DT < '2016-06-01'].to_hdf('../data/data_till_june_2016_w_descs_and_census.h5', key='data_w_descs_and_census', format='fixed')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_items] [items->None]

  f(store)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:1101: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['OnTime_Status', 'CASE_STATUS', 'CLOSURE_REASON', 'CASE_TITLE', 'SUBJECT', 'REASON', 'TYPE', 'QUEUE', 'Department', 'SubmittedPhoto', 'ClosedPhoto', 'Location', 'pwd_district', 'police_district', 'neighborhood',

## Make smaller pkl for June 2015 - June 2016 data

In [11]:
df[df.OPEN_DT > '2015-06-01'][df.OPEN_DT < '2016-06-01'].to_pickle('../data/data_subset_w_descs_and_census.pkl')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


## Let's look at the rows where there was no census tract + block in Suffolk county that matched that of the latlong